# Cyclus input file generator 

Running the cells in this notebook will generate and run a series of cyclus simulations. You will need to create a new folder in the same working directory as this notebook to store the input and output files. Paramaters in the base simulation can be changed, as well as the random variation in parameters in the subsequent cell. 

In [1]:
base_simulation = {'simulation': 
                   {'control': 
                    {'duration': '365', 'startmonth': '1', 'startyear': '2021', 'decay': 'never', 'explicit_inventory': 'true'}, 
                    'archetypes': 
                    {'spec': [{'lib': 'cycamore', 'name': 'Enrichment'}, 
                              {'lib': 'cycamore', 'name': 'Reactor'}, 
                              {'lib': 'cycamore', 'name': 'Source'}, 
                              {'lib': 'cycamore', 'name': 'Sink'}, 
                              {'lib': 'agents', 'name': 'NullRegion'}, 
                              {'lib': 'agents', 'name': 'NullInst'}, 
                              {'lib': 'cycamore', 'name': 'Reactor'}, 
                              {'lib': 'cycamore', 'name': 'Separations'}]}, 
                    'facility': [{'name': 'UraniumMine', 
                                  'config': 
                                  {'Source': {'outcommod': 'c_uore', 'outrecipe': 'r_nat_u', 'throughput': '1000000'}}}, 
                                 {'name': 'LEUEnrichmentFacility', 
                                  'config': 
                                  {'Enrichment': {'feed_commod': 'c_uore', 'feed_recipe': 'r_nat_u', 'product_commod': 'c_leu', 
                                                  'tails_commod': 'c_spent_leu', 'swu_capacity': '10000', 
                                                  'max_feed_inventory': '10000000', 
                                                  'max_enrich': '0.041', 'initial_feed': '0'}}}, 
                                 {'name': 'LWR', 
                                  'config': 
                                  {'Reactor': {'fuel_incommods': {'val': 'c_leu'}, 
                                               'fuel_inrecipes': {'val': 'r_leu'}, 
                                               'fuel_outcommods': {'val': 'c_spent_leu'}, 
                                               'fuel_outrecipes': {'val': 'r_leu_spent'}, 
                                               'cycle_time': '60', 'refuel_time': '14', 
                                               'assem_size': '195', 'n_assem_core': '3', 
                                               'n_assem_batch': '1', 'power_cap': '90'}}}, 
                                 {'name': 'SpentFuelSink', 
                                  'config': 
                                  {'Sink': {'in_commods': {'val': 'c_spent_leu'}, #'max_inv_size': '1000'
                                           }}}, 
                                 {'name': 'HEUenrich', 
                                  'config': 
                                  {'Enrichment': {'feed_commod': 'c_uore', 'feed_recipe': 'r_nat_u',
                                                  'product_commod': 'c_heu', 'tails_commod': 'heu_tails', 
                                                  'swu_capacity': '10000', 
                                                  #'max_feed_inventory': '1000', 
                                                  'max_enrich': '0.9', #'initial_feed': '100'
                                                 }}},  
                                 {'name': 'HEUSink', 
                                  'config': {'Sink': {'in_commods': {'val': 'c_heu'},
                                                      "recipe_name": 'r_heu',
                                                      #'max_inv_size': '10'
                                                     }}}], 
                    'region': {'name': 'MyRegion', 
                               'config': 
                               {'NullRegion': None}, 
                               'institution': {'name': 'inst', 
                                               'initialfacilitylist': 
                                               {'entry': [{'number': 1, 'prototype': 'UraniumMine'}, 
                                                          {'number': 1, 'prototype': 'LEUEnrichmentFacility'}, 
                                                          {'number': 1, 'prototype': 'LWR'}, 
                                                          {'number': 1, 'prototype': 'SpentFuelSink'}, 
                                                          {'number': 1, 'prototype': 'HEUenrich'}, 
                                                          {'number': 1, 'prototype': 'HEUSink'}]}, 
                                               'config': {'NullInst': None}}}, 
                    'recipe': [{'name': 'r_nat_u', 'basis': 'mass', 
                                'nuclide': [{'id': '92235', 'comp': '0.00711'}, 
                                            {'id': '92238', 'comp': '0.99289'}]}, 
                               {'name': 'r_heu', 'basis': 'mass', 
                                'nuclide': [{'id': '92235', 'comp': '0.9'}, 
                                            {'id': '92238', 'comp': '0.1'}]}, 
                               {'name': 'r_leu', 'basis': 'mass', 
                                'nuclide': [{'id': '92235', 'comp': '0.04'}, 
                                            {'id': '92238', 'comp': '0.96'}]}, 
                               {'name': 'r_leu_spent', 'basis': 'mass', 
                                'nuclide': [{'id': '92235', 'comp': '0.01'}, 
                                            {'id': '92238', 'comp': '0.94'}, 
                                            {'id': '94239', 'comp': '0.01'}, 
                                            {'id': '55135', 'comp': '0.3'}]}]}}

In [2]:
import random 

num_runs = 10
directory_label = "small_time_step"

for n in range(num_runs): 
    print(str(n+1) + " of " + str(num_runs) + " runs")
    
    sim = base_simulation.copy()
    
    diversion = random.choice([True, False])
    print(diversion)
    if not diversion: 
        sim["simulation"]["facility"] = sim["simulation"]["facility"][0:4]
        sim["simulation"]["region"]["institution"]["initialfacilitylist"]["entry"] = sim["simulation"]["region"]["institution"]["initialfacilitylist"]["entry"][0:4]
    
    throughput = float(sim["simulation"]["facility"][0]["config"]["Source"]["throughput"]) * random.uniform(0.8, 1.2)
    cycletime = int(sim["simulation"]["facility"][2]["config"]["Reactor"]["cycle_time"]) + random.randint(-15, 15)
    
    sim["simulation"]["facility"][0]["config"]["Source"]["throughput"] = str(throughput)
    sim["simulation"]["facility"][2]["config"]["Reactor"]["cycle_time"] = str(cycletime)
    
    
    
    cyclus_file = f"{directory_label}/run" + str(n) + ".py"
    with open(cyclus_file, "w") as outfile: 
        outfile.write("SIMULATION = " + str(sim))
    outfile.close()

1 of 10 runs
False
2 of 10 runs
True
3 of 10 runs
True
4 of 10 runs
True
5 of 10 runs
False
6 of 10 runs
False
7 of 10 runs
False
8 of 10 runs
False
9 of 10 runs
True
10 of 10 runs
True


In [3]:
import subprocess

for n in range(num_runs):
    process = subprocess.run(['cyclus', f'{directory_label}/run{n}.py', '-o', f'{directory_label}/run{n}.sqlite'], stdout=subprocess.PIPE)

    with open(f'small_time_step/run{n}.log', 'w') as f:
        print(process.stdout.decode('utf-8'))
        f.write(process.stdout.decode('utf-8'))

              :                                                               
          .CL:CC CC             _Q     _Q  _Q_Q    _Q    _Q              _Q   
        CC;CCCCCCCC:C;         /_\)   /_\)/_/\\)  /_\)  /_\)            /_\)  
        CCCCCCCCCCCCCl       __O|/O___O|/O_OO|/O__O|/O__O|/O____________O|/O__
     CCCCCCf     iCCCLCC     /////////////////////////////////////////////////
     iCCCt  ;;;;;.  CCCC                                                      
    CCCC  ;;;;;;;;;. CClL.                          c                         
   CCCC ,;;       ;;: CCCC  ;                   : CCCCi                       
    CCC ;;         ;;  CC   ;;:                CCC`   `C;                     
  lCCC ;;              CCCC  ;;;:             :CC .;;. C;   ;    :   ;  :;;   
  CCCC ;.              CCCC    ;;;,           CC ;    ; Ci  ;    :   ;  :  ;  
   iCC :;               CC       ;;;,        ;C ;       CC  ;    :   ; .      
  CCCi ;;               CCC        ;;;.      .C ;   

              :                                                               
          .CL:CC CC             _Q     _Q  _Q_Q    _Q    _Q              _Q   
        CC;CCCCCCCC:C;         /_\)   /_\)/_/\\)  /_\)  /_\)            /_\)  
        CCCCCCCCCCCCCl       __O|/O___O|/O_OO|/O__O|/O__O|/O____________O|/O__
     CCCCCCf     iCCCLCC     /////////////////////////////////////////////////
     iCCCt  ;;;;;.  CCCC                                                      
    CCCC  ;;;;;;;;;. CClL.                          c                         
   CCCC ,;;       ;;: CCCC  ;                   : CCCCi                       
    CCC ;;         ;;  CC   ;;:                CCC`   `C;                     
  lCCC ;;              CCCC  ;;;:             :CC .;;. C;   ;    :   ;  :;;   
  CCCC ;.              CCCC    ;;;,           CC ;    ; Ci  ;    :   ;  :  ;  
   iCC :;               CC       ;;;,        ;C ;       CC  ;    :   ; .      
  CCCi ;;               CCC        ;;;.      .C ;   

              :                                                               
          .CL:CC CC             _Q     _Q  _Q_Q    _Q    _Q              _Q   
        CC;CCCCCCCC:C;         /_\)   /_\)/_/\\)  /_\)  /_\)            /_\)  
        CCCCCCCCCCCCCl       __O|/O___O|/O_OO|/O__O|/O__O|/O____________O|/O__
     CCCCCCf     iCCCLCC     /////////////////////////////////////////////////
     iCCCt  ;;;;;.  CCCC                                                      
    CCCC  ;;;;;;;;;. CClL.                          c                         
   CCCC ,;;       ;;: CCCC  ;                   : CCCCi                       
    CCC ;;         ;;  CC   ;;:                CCC`   `C;                     
  lCCC ;;              CCCC  ;;;:             :CC .;;. C;   ;    :   ;  :;;   
  CCCC ;.              CCCC    ;;;,           CC ;    ; Ci  ;    :   ;  :  ;  
   iCC :;               CC       ;;;,        ;C ;       CC  ;    :   ; .      
  CCCi ;;               CCC        ;;;.      .C ;   